In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
import seaborn as sns
from shapely.geometry import Point, LineString, MultiPoint
import numpy as np
import contextily as ctx
import requests
from io import StringIO, BytesIO
import json
import datetime as dt
from ast import literal_eval
from shapely.wkt import loads
import plotly.express as px
from dotenv import load_dotenv, find_dotenv

# intro
This notebook is quite a mess, so I'll explain: 
We use the ```nearbysearch``` [Link](https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests) to get all the ```bars```, ```restaurants``` and ```cafe``` business hours. 

Then we fetch the unique id ```reference``` from the list of businesses and run it through the ```place_details``` API [Link](https://developers.google.com/maps/documentation/places/web-service/details)

From there we extract all the ```open``` (time-)elements and ```close``` (time-)elements and build open hour arrays stored in a 7-day dictionary. This dictionary helps creating an hourly count of open businesses. 

In the end we merge the findings with an empty time series of 2018 with an 'hourly' sequence. 


In [3]:
# API KEY
load_dotenv(find_dotenv())
GOOGLE = os.environ.get("GOOGLE")

In [73]:
def nearbysearch(GOOGLE=GOOGLE, latlng='45.05917,7.67899', radius='800'):
    """
    returns a list of businesses (restaurants, bars, cafes) within the vicinity of a given position (string).
    returns also a dictionary of all retreived data. for bugchecks.
    loops 6 times over request with next_page_token, to get 120 (6x20) results.
    """
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    params = {            
                    'location':str(latlng), #sensor
                    'radius':str(radius),
                    'type':'',
                    'key':GOOGLE,
                    'next_page_token':''
                 } 
    raw_results = {}
    all_results = []
    # get all restaurants and bars
    for _type in ['restaurant', 'bar', 'cafe']:
        print(f'searching for {_type}s')
        params['type'] = _type
        for i in range(6):
            print(f'{i}. loop')
            r = requests.get(url, params=params).json()
            params['next_page_token'] = r.get('next_page_token') #update page token
            raw_results[f'{_type}_{i}'] = r
            results = r.get('results')
            all_results = all_results + results
        params['next_page_token'] = ''
    return all_results, raw_results

In [74]:
all_, raw_ = nearbysearch()

searching for restaurants
0. loop
1. loop
2. loop
3. loop
4. loop
5. loop
searching for bars
0. loop
1. loop
2. loop
3. loop
4. loop
5. loop
searching for cafes
0. loop
1. loop
2. loop
3. loop
4. loop
5. loop


In [78]:
def get_opening(all_=all_):
    """
    returns specific opening hrs from fetched bars/restaurants given as a list. 
    """
    url = 'https://maps.googleapis.com/maps/api/place/details/json'
    params = {
        'key':GOOGLE,
        'fields':'opening_hours'
             }
    reference_set = set(business.get('reference') for business in all_)
    opening_hrs = {}
    for reference in reference_set:
        print(f'checking reference: {reference}')
        params['place_id'] = reference
        r = requests.get(url, params=params).json()
        opening_hrs[reference] = r
    
    return opening_hrs

In [79]:
opening_hrs = get_opening()

checking reference: ChIJee-P5WptiEcRjLSTAT1_xac
checking reference: ChIJSywagEJtiEcRBSKuDoA0K8I
checking reference: ChIJcQYGo2htiEcRDCBmGIryelU
checking reference: ChIJA7HHdGhtiEcRTm-KAlcNeSE
checking reference: ChIJU5OsEkJtiEcRijXq1cmSxZI
checking reference: ChIJp8GgSBVtiEcRcZhe7KgzpDM
checking reference: ChIJ_cJCMWltiEcRNhS7dtz3pLc
checking reference: ChIJ34CR-mVtiEcR4lWTsL4kaYI
checking reference: ChIJE1jruEJtiEcRCxgTBlZxD-c
checking reference: ChIJPef8W0JtiEcRX3xrOVTM628
checking reference: ChIJj7X-6kJtiEcRCxeeMgke18A
checking reference: ChIJeeINkj9tiEcRfBYot1Y8GU8
checking reference: ChIJ6bDsPkRtiEcRedpY35LwAbU
checking reference: ChIJ_cueRRVtiEcReT0ZS_x2CLI
checking reference: ChIJJ6rGVURtiEcRjVAmYHK6v3A
checking reference: ChIJmQeZs2htiEcREd1xnI9vvdw
checking reference: ChIJJ4VkrmttiEcRL80wIBiLHXU
checking reference: ChIJe76UBEJtiEcR4u6k54rXwHU
checking reference: ChIJpQPdpGdtiEcRrtDElCIN5aE
checking reference: ChIJSZgnMmZtiEcR8RIBOURbg_k
checking reference: ChIJj3LKMEJtiEcRB8wU

In [81]:
# unpack all the hours
periods_dict = {}
for reference, hrs in opening_hrs.items():
    try:
        periods = hrs.get('result').get('opening_hours').get('periods')
        if len(periods) > 1: # ignore all 24h open bars/cafes
            periods_dict[reference] = periods
    except:
        pass

In [142]:
# check one example as reference
periods_dict.get('ChIJSywagEJtiEcRBSKuDoA0K8I')

[{'close': {'day': 0, 'time': '1430'}, 'open': {'day': 0, 'time': '1200'}},
 {'close': {'day': 1, 'time': '0000'}, 'open': {'day': 0, 'time': '1830'}},
 {'close': {'day': 1, 'time': '1500'}, 'open': {'day': 1, 'time': '1230'}},
 {'close': {'day': 2, 'time': '0000'}, 'open': {'day': 1, 'time': '1800'}},
 {'close': {'day': 2, 'time': '1430'}, 'open': {'day': 2, 'time': '1200'}},
 {'close': {'day': 3, 'time': '0000'}, 'open': {'day': 2, 'time': '1830'}},
 {'close': {'day': 3, 'time': '1430'}, 'open': {'day': 3, 'time': '1200'}},
 {'close': {'day': 4, 'time': '0000'}, 'open': {'day': 3, 'time': '1830'}},
 {'close': {'day': 4, 'time': '1430'}, 'open': {'day': 4, 'time': '1200'}},
 {'close': {'day': 5, 'time': '0000'}, 'open': {'day': 4, 'time': '1830'}},
 {'close': {'day': 5, 'time': '1430'}, 'open': {'day': 5, 'time': '1200'}},
 {'close': {'day': 6, 'time': '0000'}, 'open': {'day': 5, 'time': '1830'}},
 {'close': {'day': 6, 'time': '1430'}, 'open': {'day': 6, 'time': '1200'}},
 {'close': {

In [143]:
# init master dict
master_dict = {reference:{x:[] for x in list(range(7))} for reference in periods_dict.keys()}

# loop through all periods and add opening-closing ranges
for reference, periods in periods_dict.items():
    for period in periods:
        day_open = period.get('open').get('day')
        day_close = period.get('close').get('day')
        open_ = int(period.get('open').get('time')[:-2]) # only get the hour information (not minutes)
        close_ = int(period.get('close').get('time')[:-2])
        if day_open != day_close: # handle after midnight close
            range_bf = list(range(open_, 24))
            range_af = list(range(0, close_)) # closing at 00:00 is caught from day bf
            master_dict[reference][day_open].append(range_bf)
            master_dict[reference][day_close].append(range_af)
        else:
            range_ = list(range(open_, close_+1))
            master_dict[reference][day_open].append(range_) # some business have a noon break

In [146]:
# create empty week dict of days and hrs
week_dict = {day:{hour:0 for hour in list(range(24))} for day in range(7)}
# week_dict

In [147]:
# counting all appearing hours in dict and adding to normal week hours
for reference, dict_ in master_dict.items():
    for day, ranges in dict_.items():
        if len(ranges) > 0:
            for range_ in ranges:
                for hour in range_:
                    week_dict[day][hour] += 1

In [162]:
# google week starts on sunday, need to shift
dt_days = [6,0,1,2,3,4,5]
shifted_week = {dt_days[i]:week_dict.get(i) for i in week_dict.keys()}

In [189]:
# for merging with the main df need to mold it into 2018 hourly range
range_2018 = pd.DataFrame(pd.date_range('2018-01-01', '2018-12-31', freq='h'), columns=['hour'])
range_2018['count'] = range_2018.hour.apply(lambda x: shifted_week.get(x.weekday()).get(x.hour))

In [191]:
# saving
range_2018.to_csv('raw_data/business_open_2018.csv')